## Passo 0 e 1: Instalação e Configuração Inicial

In [4]:
# =============================================================================
# Passo 0: Instalação e Configuração Inicial
# =============================================================================
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import kagglehub
import os
import glob # Importando a biblioteca glob para uma busca mais robusta

# =============================================================================
# Passo 1: Coleta e Carregamento dos Dados (COM CORREÇÃO)
# =============================================================================
print("Baixando o dataset do Kaggle...")
path = kagglehub.dataset_download("rajugc/imdb-movies-dataset-based-on-genre")
print(f"Dataset baixado em: {path}")

# --- MÉTODO ROBUSTO PARA ENCONTRAR O ARQUIVO CSV ---
# Vamos procurar por QUALQUER arquivo .csv dentro da pasta baixada e suas subpastas.
# Isso torna o código resistente a mudanças na estrutura de pastas do dataset.
# O padrão '**/*.csv' significa "qualquer pasta, qualquer subpasta, qualquer arquivo que termine com .csv"
csv_files = glob.glob(os.path.join(path, '**', '*.csv'), recursive=True)

if not csv_files:
    raise FileNotFoundError(f"Nenhum arquivo .csv encontrado no diretório baixado: {path}")

# Pega o primeiro arquivo CSV encontrado
csv_path = csv_files[0]
print(f"\nArquivo CSV encontrado em: {csv_path}")

print("\nCarregando o dataset com Pandas...")
# Agora esta linha deve funcionar sem erros
df = pd.read_csv(csv_path)

print("Dimensões iniciais do dataset:", df.shape)
print("Amostra dos dados:")
print(df.head())

Baixando o dataset do Kaggle...


100%|██████████| 53.4M/53.4M [00:00<00:00, 205MB/s]

Extracting files...


Dataset baixado em: /root/.cache/kagglehub/datasets/rajugc/imdb-movies-dataset-based-on-genre/versions/3

Arquivo CSV encontrado em: /root/.cache/kagglehub/datasets/rajugc/imdb-movies-dataset-based-on-genre/versions/3/horror.csv

Carregando o dataset com Pandas...
Dimensões iniciais do dataset: (36682, 14)
Amostra dos dados:
     movie_id          movie_name  year certificate  runtime  \
0  tt15679400  Knock at the Cabin  2023           R  100 min   
1   tt9764362            The Menu  2022           R  107 min   
2   tt8760708               M3GAN  2022       PG-13  102 min   
3  tt13051810         Viking Wolf  2022       TV-MA   97 min   
4  tt10365998       Infinity Pool  2023           R  117 min   

                       genre  rating  \
0  Horror, Mystery, Thriller     6.4   
1           Horror, Thriller     7.2   
2   Horror, Sci-Fi, Thriller     6.4   
3           Horror, Thriller     5.1   
4     Crime, Horror, Mystery     6.5   

                                         descri

## Passo 2: Pré-processamento e Preparação dos Dados (VERSÃO CORRIGIDA)

In [5]:
# =============================================================================
# Passo 2: Pré-processamento e Preparação dos Dados (VERSÃO CORRIGIDA)
# =============================================================================
print("\nIniciando o pré-processamento...")

# 2.0 (NOVO) Padronização dos Nomes das Colunas
# Esta é a melhor prática: deixar todos os nomes de colunas em um formato consistente.
# Vamos converter tudo para minúsculas e substituir espaços por underscores, se houver.
df.columns = df.columns.str.lower().str.replace(' ', '_')

print("Nomes das colunas padronizados:")
print(df.columns) # Verifique os novos nomes! Agora devem ser ['movie_name', 'genre', etc.]

# 2.1 Seleção de colunas relevantes
# Agora, usamos nossa lista original de nomes em minúsculas, mas com 'movie_name'.
# ATENÇÃO: Corrigimos 'title' para 'movie_name'
colunas_relevantes = ['movie_name', 'genre', 'description', 'rating']
df_movies = df[colunas_relevantes].copy()

# (NOVO) Vamos renomear 'movie_name' para 'title' para manter a consistência com o resto do plano
df_movies.rename(columns={'movie_name': 'title'}, inplace=True)
print("\nNomes das colunas após renomear 'movie_name' para 'title':")
print(df_movies.columns)

# 2.2 Tratamento de valores ausentes
print(f"\nValores ausentes antes do tratamento:\n{df_movies.isnull().sum()}")
df_movies.dropna(subset=['description', 'genre'], inplace=True)
print(f"\nValores ausentes depois do tratamento:\n{df_movies.isnull().sum()}")
print(f"Dimensões após remover valores ausentes: {df_movies.shape}")

# 2.3 Normalização e Criação da coluna "conteúdo"
# O resto do código agora funciona perfeitamente, pois as colunas já estão em minúsculas.
df_movies['description'] = df_movies['description'].str.lower()
df_movies['genre'] = df_movies['genre'].str.lower()
df_movies['title'] = df_movies['title'].str.strip()

df_movies['content'] = df_movies['description'] + ' ' + df_movies['genre']

print("\nAmostra da nova coluna 'content':")
print(df_movies[['title', 'content']].head())


Iniciando o pré-processamento...
Nomes das colunas padronizados:
Index(['movie_id', 'movie_name', 'year', 'certificate', 'runtime', 'genre',
       'rating', 'description', 'director', 'director_id', 'star', 'star_id',
       'votes', 'gross(in_$)'],
      dtype='object')

Nomes das colunas após renomear 'movie_name' para 'title':
Index(['title', 'genre', 'description', 'rating'], dtype='object')

Valores ausentes antes do tratamento:
title              0
genre              0
description        0
rating         15523
dtype: int64

Valores ausentes depois do tratamento:
title              0
genre              0
description        0
rating         15523
dtype: int64
Dimensões após remover valores ausentes: (36682, 4)

Amostra da nova coluna 'content':
                title                                            content
0  Knock at the Cabin  while vacationing, a girl and her parents are ...
1            The Menu  a young couple travels to a remote island to e...
2               M3GA

## Passo 3 - Vetorização com TF-IDF

In [6]:
# =============================================================================
# Passo 3: Vetorização com TF-IDF (Transformando texto em números)
# =============================================================================
print("\nCriando a matriz TF-IDF...")

# Inicializa o TfidfVectorizer.
# - stop_words='english': remove palavras comuns como 'the', 'a', 'is'.
# - max_features=5000: considera apenas as 5000 palavras mais frequentes para manter a matriz gerenciável.
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Aplica o TF-IDF na nossa coluna 'content'
tfidf_matrix = tfidf.fit_transform(df_movies['content'])

print("Dimensões da matriz TF-IDF:", tfidf_matrix.shape)


Criando a matriz TF-IDF...
Dimensões da matriz TF-IDF: (36682, 5000)


## Passo 4: Implementação do KNN

In [7]:
# =============================================================================
# Passo 4: Implementação do KNN (Encontrando os vizinhos mais próximos)
# =============================================================================
print("\nTreinando o modelo KNN para busca de similaridade...")

# Inicializa o NearestNeighbors.
# - n_neighbors=6: queremos o filme original + 5 recomendações.
# - metric='cosine': a similaridade de cosseno é excelente para dados de texto (vetores TF-IDF).
# - algorithm='brute': usa uma busca de força bruta. Para datasets maiores, 'kd_tree' ou 'ball_tree' podem ser mais eficientes.
nn_model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')

# "Treina" o modelo, que na verdade apenas indexa os dados da matriz TF-IDF para buscas rápidas.
nn_model.fit(tfidf_matrix)


Treinando o modelo KNN para busca de similaridade...


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

## Passo 5: Construção da Função de Recomendação

In [8]:
# =============================================================================
# Passo 5: Construção da Função de Recomendação (O coração do sistema)
# =============================================================================
# Criamos um mapeamento do título do filme para o índice do dataframe para buscas rápidas.
# Isso evita ter que procurar pelo título toda vez.
indices = pd.Series(df_movies.index, index=df_movies['title']).drop_duplicates()

def get_recommendations(title: str):
    """
    Função que recebe o título de um filme e retorna 5 filmes similares.
    """
    # 1. Obter o índice do filme a partir do seu título
    if title not in indices:
        return f"Erro: Filme '{title}' não encontrado no dataset."

    idx = indices[title]

    # 2. Obter o vetor TF-IDF do filme selecionado
    movie_vector = tfidf_matrix[idx]

    # 3. Usar o modelo KNN para encontrar os 6 filmes mais próximos (incluindo ele mesmo)
    distances, movie_indices = nn_model.kneighbors(movie_vector)

    # 4. Pegar os índices dos 5 filmes mais similares (ignorando o primeiro, que é o próprio filme)
    # A função flatten() transforma o array 2D em 1D
    similar_movie_indices = movie_indices.flatten()[1:]

    # 5. Retornar os títulos dos filmes recomendados
    recommended_movies = df_movies.iloc[similar_movie_indices]

    return recommended_movies[['title', 'genre', 'rating', 'description']]


## Passo 6: Teste do Sistema de Reomendação

In [9]:
# =============================================================================
# Passo 6: Teste do Sistema de Recomendação (Escopo do MVP)
# =============================================================================
print("\n--- TESTANDO O SISTEMA DE RECOMENDAÇÃO ---")

# Exemplo 1: Um filme de super-herói
movie_choice = "The Dark Knight"
print(f"\nRecomendações para '{movie_choice}':")
recommendations = get_recommendations(movie_choice)
print(recommendations)

# Exemplo 2: Um filme de ficção científica
movie_choice = "Inception"
print(f"\nRecomendações para '{movie_choice}':")
recommendations = get_recommendations(movie_choice)
print(recommendations)

# Exemplo 3: Uma animação
movie_choice = "Spirited Away"
print(f"\nRecomendações para '{movie_choice}':")
recommendations = get_recommendations(movie_choice)
print(recommendations)

# Exemplo 4: Teste de um filme que não existe
movie_choice = "Meu Filme Favorito Que Nao Existe"
print(f"\nRecomendações para '{movie_choice}':")
recommendations = get_recommendations(movie_choice)
print(recommendations)


--- TESTANDO O SISTEMA DE RECOMENDAÇÃO ---

Recomendações para 'The Dark Knight':
Erro: Filme 'The Dark Knight' não encontrado no dataset.

Recomendações para 'Inception':
Erro: Filme 'Inception' não encontrado no dataset.

Recomendações para 'Spirited Away':
Erro: Filme 'Spirited Away' não encontrado no dataset.

Recomendações para 'Meu Filme Favorito Que Nao Existe':
Erro: Filme 'Meu Filme Favorito Que Nao Existe' não encontrado no dataset.


## Passo 7: Exportando o Modelo KNN e a Matriz TF-IDF

Para poder reutilizar o sistema de recomendação sem ter que re-treinar o modelo toda vez, vamos salvar o modelo `nn_model` e a matriz `tfidf_matrix` em arquivos. Usaremos a biblioteca `joblib`, que é eficiente para salvar objetos NumPy e scikit-learn.

In [10]:
import joblib
import os

# Define os nomes dos arquivos para salvar
model_filename = 'knn_model.joblib'
tfidf_matrix_filename = 'tfidf_matrix.joblib'
indices_filename = 'indices.joblib' # Também é útil salvar os índices para carregar rapidamente

print(f"\nSalvando o modelo KNN em: {model_filename}")
# Salva o modelo KNN treinado
joblib.dump(nn_model, model_filename)
print("Modelo KNN salvo com sucesso.")

print(f"\nSalvando a matriz TF-IDF em: {tfidf_matrix_filename}")
# Salva a matriz TF-IDF
joblib.dump(tfidf_matrix, tfidf_matrix_filename)
print("Matriz TF-IDF salva com sucesso.")

print(f"\nSalvando os índices em: {indices_filename}")
# Salva os índices
joblib.dump(indices, indices_filename)
print("Índices salvos com sucesso.")

print("\nVocê pode encontrar os arquivos salvos no diretório do notebook.")


Salvando o modelo KNN em: knn_model.joblib
Modelo KNN salvo com sucesso.

Salvando a matriz TF-IDF em: tfidf_matrix.joblib
Matriz TF-IDF salva com sucesso.

Salvando os índices em: indices.joblib
Índices salvos com sucesso.

Você pode encontrar os arquivos salvos no diretório do notebook.


## Passo 8: Exemplo de Como Carregar e Usar o Modelo Salvo

Agora, para usar o sistema de recomendação em outro notebook ou sessão, você precisaria carregar esses arquivos. Veja um exemplo de como fazer isso:

In [11]:
import joblib
import pandas as pd

# Define os nomes dos arquivos salvos
model_filename = 'knn_model.joblib'
tfidf_matrix_filename = 'tfidf_matrix.joblib'
indices_filename = 'indices.joblib'

print("\nCarregando o modelo KNN, matriz TF-IDF e índices dos arquivos...")
# Carrega o modelo KNN
loaded_nn_model = joblib.load(model_filename)
print("Modelo KNN carregado com sucesso.")

# Carrega a matriz TF-IDF
loaded_tfidf_matrix = joblib.load(tfidf_matrix_filename)
print("Matriz TF-IDF carregada com sucesso.")

# Carrega os índices
loaded_indices = joblib.load(indices_filename)
print("Índices carregados com sucesso.")

# IMPORTANTE: Para usar a função get_recommendations, você também precisará do df_movies
# Você pode salvar o df_movies em um arquivo (por exemplo, CSV ou pickle) e carregá-lo aqui,
# OU recriar o df_movies a partir dos dados originais se for rápido o suficiente.
# Para este exemplo, vamos assumir que df_movies já foi carregado ou recriado.
# Se df_movies não existir, a próxima linha falhará.

# Recriando uma versão mínima de df_movies apenas com as colunas necessárias para a função
# Em um cenário real, você carregaria o df_movies completo ou um subconjunto relevante.
# Para este exemplo de carregamento, vamos apenas criar um DataFrame dummy que se pareça com df_movies
# para que a função de recomendação possa ser testada.
# **Em um caso de uso real, você carregaria o df_movies salvo.**
# Exemplo: loaded_df_movies = pd.read_pickle('df_movies.pkl')

# Para o propósito de DEMONSTRAR o carregamento:
# Precisamos do DataFrame original (ou pelo menos as colunas 'title', 'genre', 'rating', 'description')
# para que a função get_recommendations possa buscar os detalhes dos filmes recomendados.
# Se você não salvou o df_movies, você precisará carregá-lo ou recriá-lo.
# Vamos usar o df_movies que já está na sessão para este teste.
# Em um cenário "do zero" (novo notebook), você carregaria o df_movies salvo.

def get_recommendations_from_loaded(title: str):
    """
    Função que recebe o título de um filme e retorna 5 filmes similares
    usando os objetos carregados.
    """
    # 1. Obter o índice do filme a partir do seu título usando os índices carregados
    if title not in loaded_indices:
        return f"Erro: Filme '{title}' não encontrado no dataset carregado."

    idx = loaded_indices[title]

    # 2. Obter o vetor TF-IDF do filme selecionado da matriz carregada
    movie_vector = loaded_tfidf_matrix[idx]

    # 3. Usar o modelo KNN carregado para encontrar os 6 filmes mais próximos
    distances, movie_indices = loaded_nn_model.kneighbors(movie_vector)

    # 4. Pegar os índices dos 5 filmes mais similares (ignorando o primeiro)
    similar_movie_indices = movie_indices.flatten()[1:]

    # 5. Retornar os títulos e detalhes dos filmes recomendados usando o df_movies original (ou carregado)
    # **Importante:** Nesta demonstração, estamos usando o 'df_movies' da sessão atual.
    # Em um cenário real de carregamento, você usaria um 'loaded_df_movies'.
    recommended_movies = df_movies.iloc[similar_movie_indices] # Use df_movies da sessão atual

    return recommended_movies[['title', 'genre', 'rating', 'description']]


print("\n--- TESTANDO O SISTEMA DE RECOMENDAÇÃO CARREGADO ---")

# Exemplo de teste com o modelo carregado
movie_choice_loaded = "The Dark Knight" # Use um filme que sabemos que está no dataset
print(f"\nRecomendações (carregadas) para '{movie_choice_loaded}':")
recommendations_loaded = get_recommendations_from_loaded(movie_choice_loaded)
print(recommendations_loaded)

# Exemplo de teste com um filme que não existe (no dataset carregado)
movie_choice_loaded_fail = "Um Filme Inexistente Para Teste"
print(f"\nRecomendações (carregadas) para '{movie_choice_loaded_fail}':")
recommendations_loaded_fail = get_recommendations_from_loaded(movie_choice_loaded_fail)
print(recommendations_loaded_fail)


Carregando o modelo KNN, matriz TF-IDF e índices dos arquivos...
Modelo KNN carregado com sucesso.
Matriz TF-IDF carregada com sucesso.
Índices carregados com sucesso.

--- TESTANDO O SISTEMA DE RECOMENDAÇÃO CARREGADO ---

Recomendações (carregadas) para 'The Dark Knight':
Erro: Filme 'The Dark Knight' não encontrado no dataset carregado.

Recomendações (carregadas) para 'Um Filme Inexistente Para Teste':
Erro: Filme 'Um Filme Inexistente Para Teste' não encontrado no dataset carregado.


In [12]:
# Volte para o diretório onde os arquivos joblib foram salvos
# Se você não tem certeza onde eles foram salvos, pode usar !ls para listar os arquivos
# ou !pwd para ver o diretório atual.
# Por padrão, eles devem estar no diretório de conteúdo do Colab (/content/).
# Se você usou 'cd ..', você pode precisar voltar para '/content/'
import os
os.chdir('/content/')

print("Diretório atual:", os.getcwd())

# Tente executar a célula ed41e9e0 novamente AGORA

Diretório atual: /content


In [13]:
# Especifique o nome do arquivo CSV de saída
output_csv_filename = 'df_movies.csv'

print(f"Exportando o DataFrame 'df_movies' para '{output_csv_filename}'...")

# Use o método .to_csv() para salvar o DataFrame
# index=False impede que o índice do DataFrame seja escrito como uma coluna no CSV
df_movies.to_csv(output_csv_filename, index=False)

print(f"DataFrame exportado com sucesso para '{output_csv_filename}'.")

# Opcional: Verificar se o arquivo foi criado
import os
if os.path.exists(output_csv_filename):
    print(f"O arquivo '{output_csv_filename}' foi encontrado no diretório atual.")
else:
    print(f"O arquivo '{output_csv_filename}' NÃO foi encontrado. Algo pode ter dado errado.")

Exportando o DataFrame 'df_movies' para 'df_movies.csv'...
DataFrame exportado com sucesso para 'df_movies.csv'.
O arquivo 'df_movies.csv' foi encontrado no diretório atual.
